# Instacart Market Basket Analysis

## Index
1. Extract the data
2. Understand the data
3. Preprocesssing stage
4. Training different models to set a Benchmark
5. Improvise the Model and Repeat
6. Predict on Test data.

In [29]:
import numpy as np
import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [30]:
import os
# List all the data we have.
print(os.listdir('data'))

['.DS_Store', 'aisles.csv', 'departments.csv', 'order_products__prior.csv', 'order_products__train.csv', 'orders.csv', 'products.csv', 'sample_submission.csv']


# 1. Extract the Data

In [6]:
aisles      = pd.read_csv('data/aisles.csv')
departments = pd.read_csv('data/departments.csv')
order_prior = pd.read_csv('data/order_products__prior.csv')
order_train = pd.read_csv('data/order_products__train.csv')
orders      = pd.read_csv('data/orders.csv')
products    = pd.read_csv('data/products.csv')

In [50]:
print(aisles.shape)
print(aisles.head())

(134, 2)
   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation


In [51]:
print(departments.shape)
print(departments.head())

(21, 2)
   department_id department
0              1     frozen
1              2      other
2              3     bakery
3              4    produce
4              5    alcohol


In [69]:
print (order_prior.shape)
print(order_prior.sort_values('order_id').head(10))

(32434489, 4)
    order_id  product_id  add_to_cart_order  reordered
0          2       33120                  1          1
1          2       28985                  2          1
2          2        9327                  3          0
3          2       45918                  4          1
4          2       30035                  5          0
5          2       17794                  6          1
6          2       40141                  7          1
7          2        1819                  8          1
8          2       43668                  9          0
16         3       32665                  8          1


In [84]:
print(order_train.shape)
print(order_train[ order_train['order_id']==1] )

(1384617, 4)
   order_id  product_id  add_to_cart_order  reordered
0         1       49302                  1          1
1         1       11109                  2          1
2         1       10246                  3          0
3         1       49683                  4          0
4         1       43633                  5          1
5         1       13176                  6          0
6         1       47209                  7          0
7         1       22035                  8          1


In [87]:
print(orders.shape,'\n',orders.head())

(3421083, 7) 
    order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  days_since_prior_order
0   2539329        1    prior             1          2                  8                     NaN
1   2398795        1    prior             2          3                  7                    15.0
2    473747        1    prior             3          3                 12                    21.0
3   2254736        1    prior             4          4                  7                    29.0
4    431534        1    prior             5          4                 15                    28.0
(3214874, 7)
(131209, 7)
(75000, 7)


In [73]:
print(products.shape)
print(products.head())

(49688, 4)
   product_id                                       product_name  aisle_id  department_id
0           1                         Chocolate Sandwich Cookies        61             19
1           2                                   All-Seasons Salt       104             13
2           3               Robust Golden Unsweetened Oolong Tea        94              7
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38              1
4           5                          Green Chile Anytime Sauce         5             13
